In [0]:
# LTFS Data Science FinHack 2
## https://datahack.analyticsvidhya.com/contest/ltfs-data-science-finhack-2-an-online-hackathon/

In [0]:
# Simple approach using Ada Boost

In [0]:
%reload_ext autoreload
%autoreload 2

In [0]:
# importing Libraries
import os
import datetime
from tqdm import tqdm
## importing Fastai library for add_datepart function
from fastai import *
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor

In [140]:
# Fixing Seed=2
manualSeed = 2

np.random.seed(manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

In [141]:
# Mounting Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [142]:
path="/content/gdrive/My Drive/AIML/Data Science FinHack 2/"
print(os.listdir(path))

['My_Dataset', 'My_Notebooks', 'My_Models', 'My_Outputs', 'Bin']


In [0]:
# Function to convert date to pandas datetime datatype
def cnv_2_datetime(df,col_name='application_date'):
  df[col_name] = pd.to_datetime(df[col_name], format='%d-%m-%Y')
  return df

In [0]:
# Function to convert categorical columns as pandas category type
def cnv_2_category(df,col_names):
  for col in col_names:
    df[col] = df[col].astype('category')
  return df

In [0]:
# To drop columns
def drop_col(df,col_names):
  df.drop(col_names, axis=1,inplace=True)
  return df

In [0]:
# All above functions together
def process_df(df,cat_names,drop_cols,col_name='application_date'):
  df[col_name] = pd.to_datetime(df[col_name], format='%d-%m-%Y')
  for col in cat_names:
    df[col] = df[col].astype('category')
  df.drop(drop_cols, axis=1,inplace=True)
  return df

In [147]:
# Loading Datasets
# Manually splitted .csv files into train and validation files from original files
# Created test files from test .csv file to match train and validation sets format

train_df_1 = pd.read_csv(path+'My_Dataset/'+'train_Seg_1.csv')
valid_df_1 = pd.read_csv(path+'My_Dataset/'+'valid_Seg_1.csv')
test_df_1 = pd.read_csv(path+'My_Dataset/'+'test_Seg_1.csv')

train_df_2 = pd.read_csv(path+'My_Dataset/'+'train_Seg_2.csv')
valid_df_2 = pd.read_csv(path+'My_Dataset/'+'valid_Seg_2.csv')
test_df_2 = pd.read_csv(path+'My_Dataset/'+'test_Seg_2.csv')

sub_df = pd.read_csv(path+'My_Dataset/'+'sample_submission_IIzFVsf.csv')

# Copying test for final submission
pred_test_df_1=test_df_1.copy(deep=True)
pred_test_df_2=test_df_2.copy(deep=True)
train_df_1.head(2)

,application_date,segment,branch_id,state,zone,case_count
0,01-04-2017,1,1,WEST BENGAL,EAST,40
1,03-04-2017,1,1,WEST BENGAL,EAST,5


In [0]:
dfs_1=[train_df_1,valid_df_1,test_df_1]
dfs_2=[train_df_2,valid_df_2,test_df_2]

In [0]:
for df in dfs_1:
  df=cnv_2_datetime(df,col_name='application_date')
for df in dfs_2:
  df=cnv_2_datetime(df,col_name='application_date')

In [0]:
# Added Share price data from Yahoo finance
# https://finance.yahoo.com/quote/L%26TFH.NS/history?p=L%26TFH.NS

ltfs_share = pd.read_csv(path+'My_Dataset/'+'LTFS_Share_Price_Data.csv')
ltfs_share = cnv_2_datetime(ltfs_share,col_name='Date')

In [0]:
# Merging Share data with train, validation and test
train_df_1=train_df_1.merge(ltfs_share, how='left', left_on='application_date', right_on='Date')
valid_df_1=valid_df_1.merge(ltfs_share, how='left', left_on='application_date', right_on='Date')
test_df_1=test_df_1.merge(ltfs_share, how='left', left_on='application_date', right_on='Date')
train_df_2=train_df_2.merge(ltfs_share, how='left', left_on='application_date', right_on='Date')
valid_df_2=valid_df_2.merge(ltfs_share, how='left', left_on='application_date', right_on='Date')
test_df_2=test_df_2.merge(ltfs_share, how='left', left_on='application_date', right_on='Date')

In [0]:
train_df_1.fillna(0, inplace=True)
valid_df_1.fillna(0, inplace=True)
test_df_1.fillna(0, inplace=True)
train_df_2.fillna(0, inplace=True)
valid_df_2.fillna(0, inplace=True)
test_df_2.fillna(0, inplace=True)

In [153]:
train_df_1.head()

,application_date,segment,branch_id,state,zone,case_count,Date,Open,Close,Volume
0,2017-04-01,1,1,WEST BENGAL,EAST,40,0,0.000000,0.000000,0.0
1,2017-04-03,1,1,WEST BENGAL,EAST,5,2017-04-03 00:00:00,122.650002,123.500000,2446127.0
2,2017-04-04,1,1,WEST BENGAL,EAST,4,0,0.000000,0.000000,0.0
3,2017-04-05,1,1,WEST BENGAL,EAST,113,2017-04-05 00:00:00,123.500000,123.900002,2524567.0
4,2017-04-07,1,1,WEST BENGAL,EAST,76,2017-04-07 00:00:00,122.800003,120.800003,3097966.0


In [0]:
# Adding holidays data
# https://www.calendarlabs.com/holidays/india/2019

with open(path+'My_Dataset/'+'Holidays.txt', 'r') as f:
    x = f.readlines()

In [0]:
Holiday_list=[]
for i in x:
  Holiday_list.append(i.rstrip('\n'))
#Holiday_list

In [0]:
# Getting columns for pre-processing
cat_col_1=['branch_id','state','zone']
cat_col_2=['state']
drop_cols_1=['segment','Date']
drop_cols_2=['segment','Date','branch_id','zone']
pred_drop_cols_1=['segment']
pred_drop_cols_2=['segment','branch_id','zone']

In [0]:
train_df_1 = process_df(train_df_1,cat_col_1,drop_cols_1)
valid_df_1 = process_df(valid_df_1,cat_col_1,drop_cols_1)

test_df_1 = process_df(test_df_1,cat_col_1,drop_cols_1)

pred_test_df_1 = process_df(pred_test_df_1,cat_col_1,pred_drop_cols_1)


In [0]:
train_df_2 = process_df(train_df_2,cat_col_2,drop_cols_2)
valid_df_2 = process_df(valid_df_2,cat_col_2,drop_cols_2)

test_df_2 = process_df(test_df_2,cat_col_2,drop_cols_2)

pred_test_df_2 = process_df(pred_test_df_2,cat_col_2,pred_drop_cols_2)

In [0]:
def add_holidays(df,Holiday_list=Holiday_list):
  df["is_holiday"] = (df.application_date.astype("str").isin(Holiday_list)).astype(int)
  return df

In [0]:
train_df_1 = add_holidays(train_df_1)
valid_df_1 = add_holidays(valid_df_1)
test_df_1 = add_holidays(test_df_1)

train_df_2 = add_holidays(train_df_2)
valid_df_2 = add_holidays(valid_df_2)
test_df_2 = add_holidays(test_df_2)

In [161]:
train_df_1.dtypes

application_date    datetime64[ns]
branch_id                 category
state                     category
zone                      category
case_count                   int64
Open                       float64
Close                      float64
Volume                     float64
is_holiday                   int64
dtype: object

In [162]:
train_df_1.head()

,application_date,branch_id,state,zone,case_count,Open,Close,Volume,is_holiday
0,2017-04-01,1,WEST BENGAL,EAST,40,0.000000,0.000000,0.0,0
1,2017-04-03,1,WEST BENGAL,EAST,5,122.650002,123.500000,2446127.0,0
2,2017-04-04,1,WEST BENGAL,EAST,4,0.000000,0.000000,0.0,0
3,2017-04-05,1,WEST BENGAL,EAST,113,123.500000,123.900002,2524567.0,1
4,2017-04-07,1,WEST BENGAL,EAST,76,122.800003,120.800003,3097966.0,0


In [163]:
# Adding date time features

add_datepart(train_df_1,'application_date',drop=True,time=False)
add_datepart(valid_df_1,'application_date',drop=True,time=False)

add_datepart(test_df_1,'application_date',drop=True,time=False)

add_datepart(train_df_2,'application_date',drop=True,time=False)
add_datepart(valid_df_2,'application_date',drop=True,time=False)

add_datepart(test_df_2,'application_date',drop=True,time=False)

,state,Open,Close,Volume,is_holiday,application_Year,application_Month,application_Week,application_Day,application_Dayofweek,application_Dayofyear,application_Is_month_end,application_Is_month_start,application_Is_quarter_end,application_Is_quarter_start,application_Is_year_end,application_Is_year_start,application_Elapsed
0,ASSAM,107.349998,102.699997,11777944.0,0,2019,7,30,24,2,205,False,False,False,False,False,False,1563926400
1,ASSAM,103.050003,102.000000,8515821.0,0,2019,7,30,25,3,206,False,False,False,False,False,False,1564012800
2,ASSAM,102.199997,103.750000,6655041.0,0,2019,7,30,26,4,207,False,False,False,False,False,False,1564099200
3,ASSAM,0.000000,0.000000,0.0,0,2019,7,30,27,5,208,False,False,False,False,False,False,1564185600
4,ASSAM,0.000000,0.000000,0.0,0,2019,7,30,28,6,209,False,False,False,False,False,False,1564272000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483,WEST BENGAL,0.000000,0.000000,0.0,0,2019,10,42,20,6,293,False,False,False,False,False,False,1571529600
1484,WEST BENGAL,0.000000,0.000000,0.0,0,2019,10,43,21,0,294,False,False,False,False,False,False,1571616000
1485,WEST BENGAL,84.099998,86.599998,23587540.0,0,2019,10,43,22,1,295,False,False,False,False,False,False,1571702400
1486,WEST BENGAL,86.500000,87.650002,9826408.0,0,2019,10,43,23,2,296,False,False,False,False,False,False,1571788800


In [164]:
train_df_1.head()

,branch_id,state,zone,case_count,Open,Close,Volume,is_holiday,application_Year,application_Month,application_Week,application_Day,application_Dayofweek,application_Dayofyear,application_Is_month_end,application_Is_month_start,application_Is_quarter_end,application_Is_quarter_start,application_Is_year_end,application_Is_year_start,application_Elapsed
0,1,WEST BENGAL,EAST,40,0.000000,0.000000,0.0,0,2017,4,13,1,5,91,False,True,False,True,False,False,1491004800
1,1,WEST BENGAL,EAST,5,122.650002,123.500000,2446127.0,0,2017,4,14,3,0,93,False,False,False,False,False,False,1491177600
2,1,WEST BENGAL,EAST,4,0.000000,0.000000,0.0,0,2017,4,14,4,1,94,False,False,False,False,False,False,1491264000
3,1,WEST BENGAL,EAST,113,123.500000,123.900002,2524567.0,1,2017,4,14,5,2,95,False,False,False,False,False,False,1491350400
4,1,WEST BENGAL,EAST,76,122.800003,120.800003,3097966.0,0,2017,4,14,7,4,97,False,False,False,False,False,False,1491523200


In [0]:
# Encoding labels for categorical variables
le=LabelEncoder()
train_df_1['state']=le.fit_transform(train_df_1['state'].astype('str'))
train_df_1['zone']=le.fit_transform(train_df_1['zone'].astype('str'))
train_df_1['branch_id']=le.fit_transform(train_df_1['branch_id'].astype('str'))

valid_df_1['state']=le.fit_transform(valid_df_1['state'].astype('str'))
valid_df_1['zone']=le.fit_transform(valid_df_1['zone'].astype('str'))
valid_df_1['branch_id']=le.fit_transform(valid_df_1['branch_id'].astype('str'))

test_df_1['state']=le.fit_transform(test_df_1['state'].astype('str'))
test_df_1['zone']=le.fit_transform(test_df_1['zone'].astype('str'))
test_df_1['branch_id']=le.fit_transform(test_df_1['branch_id'].astype('str'))

train_df_2['state']=le.fit_transform(train_df_2['state'].astype('str'))
valid_df_2['state']=le.fit_transform(valid_df_2['state'].astype('str'))
test_df_2['state']=le.fit_transform(test_df_2['state'].astype('str'))

In [166]:
train_df_1.head()

,branch_id,state,zone,case_count,Open,Close,Volume,is_holiday,application_Year,application_Month,application_Week,application_Day,application_Dayofweek,application_Dayofyear,application_Is_month_end,application_Is_month_start,application_Is_quarter_end,application_Is_quarter_start,application_Is_year_end,application_Is_year_start,application_Elapsed
0,0,19,1,40,0.000000,0.000000,0.0,0,2017,4,13,1,5,91,False,True,False,True,False,False,1491004800
1,0,19,1,5,122.650002,123.500000,2446127.0,0,2017,4,14,3,0,93,False,False,False,False,False,False,1491177600
2,0,19,1,4,0.000000,0.000000,0.0,0,2017,4,14,4,1,94,False,False,False,False,False,False,1491264000
3,0,19,1,113,123.500000,123.900002,2524567.0,1,2017,4,14,5,2,95,False,False,False,False,False,False,1491350400
4,0,19,1,76,122.800003,120.800003,3097966.0,0,2017,4,14,7,4,97,False,False,False,False,False,False,1491523200


In [167]:
train_df_2.head()

,state,case_count,Open,Close,Volume,is_holiday,application_Year,application_Month,application_Week,application_Day,application_Dayofweek,application_Dayofyear,application_Is_month_end,application_Is_month_start,application_Is_quarter_end,application_Is_quarter_start,application_Is_year_end,application_Is_year_start,application_Elapsed
0,0,0,0.000000,0.000000,0.0,0,2017,4,13,1,5,91,False,True,False,True,False,False,1491004800
1,0,0,0.000000,0.000000,0.0,0,2017,4,13,2,6,92,False,False,False,False,False,False,1491091200
2,0,0,122.650002,123.500000,2446127.0,0,2017,4,14,3,0,93,False,False,False,False,False,False,1491177600
3,0,0,0.000000,0.000000,0.0,0,2017,4,14,4,1,94,False,False,False,False,False,False,1491264000
4,0,0,123.500000,123.900002,2524567.0,1,2017,4,14,5,2,95,False,False,False,False,False,False,1491350400


In [0]:
y1=train_df_1['case_count']
y2=train_df_2['case_count']

In [0]:
yv1=valid_df_1['case_count']
yv2=valid_df_2['case_count']

In [0]:
X1=train_df_1.drop(columns=['case_count'])
X2=train_df_2.drop(columns=['case_count'])

In [0]:
Xv1=valid_df_1.drop(columns=['case_count'])
Xv2=valid_df_2.drop(columns=['case_count'])

In [172]:
X1.head()

,branch_id,state,zone,Open,Close,Volume,is_holiday,application_Year,application_Month,application_Week,application_Day,application_Dayofweek,application_Dayofyear,application_Is_month_end,application_Is_month_start,application_Is_quarter_end,application_Is_quarter_start,application_Is_year_end,application_Is_year_start,application_Elapsed
0,0,19,1,0.000000,0.000000,0.0,0,2017,4,13,1,5,91,False,True,False,True,False,False,1491004800
1,0,19,1,122.650002,123.500000,2446127.0,0,2017,4,14,3,0,93,False,False,False,False,False,False,1491177600
2,0,19,1,0.000000,0.000000,0.0,0,2017,4,14,4,1,94,False,False,False,False,False,False,1491264000
3,0,19,1,123.500000,123.900002,2524567.0,1,2017,4,14,5,2,95,False,False,False,False,False,False,1491350400
4,0,19,1,122.800003,120.800003,3097966.0,0,2017,4,14,7,4,97,False,False,False,False,False,False,1491523200


In [173]:
y1.head()

0     40
1      5
2      4
3    113
4     76
Name: case_count, dtype: int64

In [174]:
col=X1.columns.to_list()
col

['branch_id',
 'state',
 'zone',
 'Open',
 'Close',
 'Volume',
 'is_holiday',
 'application_Year',
 'application_Month',
 'application_Week',
 'application_Day',
 'application_Dayofweek',
 'application_Dayofyear',
 'application_Is_month_end',
 'application_Is_month_start',
 'application_Is_quarter_end',
 'application_Is_quarter_start',
 'application_Is_year_end',
 'application_Is_year_start',
 'application_Elapsed']

In [0]:
def mape(y_true, y_pred):
  if len(y_true)==len(y_pred):
    y_true=y_true.astype(float)
    y_pred=y_pred.astype(float)
    y_true[y_true==0]=0.00001
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
  else:
    raise Exception('Length of inputs did not match')

In [176]:
# Using Adaboost Regressor for segment 1

ada_1 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=17), n_estimators=300, loss='exponential')
ada_1.fit(X1, y1)
yp1=ada_1.predict(Xv1)
metric=mean_absolute_error(yv1, yp1)
metric
#dep=17, n=300 ---> 11.929

11.646630772333875

In [177]:
# Using Adaboost Regressor for segment 2

ada_2 = AdaBoostRegressor(DecisionTreeRegressor(max_depth=22), n_estimators=300, loss='exponential')
ada_2.fit(X2, y2)
yp2=ada_2.predict(Xv2)
metric=mean_absolute_error(yv2, yp2)
metric
#dep=22, n=300 ---> 152.79

153.21530337325098

In [0]:
# Making predictions
preds1 = ada_1.predict(test_df_1)
preds2 = ada_2.predict(test_df_2)

In [0]:
# Summing up predictions datewise
pred_test_df_1['preds']=preds1.astype(int)
pred_test_df_2['preds']=preds2.astype(int)
seg_1_preds=pred_test_df_1.groupby(['application_date']).sum()
seg_2_preds=pred_test_df_2.groupby(['application_date']).sum()
seg_1_preds.reset_index(drop=True, inplace=True)
seg_2_preds.reset_index(drop=True, inplace=True)

In [0]:
# Predictions to submission file
sub_df_pred = pd.concat([seg_1_preds, seg_2_preds])
sub_df['case_count']=sub_df_pred['preds'].values
sub_df.to_csv(path+'My_Outputs'+'/'+'Submission_Ada_Boost_Share_Holidays.csv',index=False)